# Prototyping LangChain Application with Production Minded Changes

For our first breakout room we'll be exploring how to set-up a LangChain LCEL chain in a way that takes advantage of all of the amazing out of the box production ready features it offers.

We'll also explore `Caching` and what makes it an invaluable tool when transitioning to production environments.


## Task 1: Dependencies and Set-Up

Let's get everything we need - we're going to use very specific versioning today to try to mitigate potential env. issues!

> NOTE: Dependency issues are a large portion of what you're going to be tackling as you integrate new technology into your work - please keep in mind that one of the things you should be passively learning throughout this course is ways to mitigate dependency issues.

In [24]:
!pip install -qU langchain_openai==0.2.0 langchain_community==0.3.0 langchain==0.3.0 pymupdf==1.24.10 qdrant-client==1.11.2 langchain_qdrant==0.1.4 langsmith==0.1.121

We'll need an OpenAI API Key:

In [1]:
!pip install python-dotenv

In [2]:
from dotenv import load_dotenv; _ = load_dotenv()

In [1]:
# import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


And the LangSmith set-up:

In [5]:
import uuid

os.environ["LANGCHAIN_PROJECT"] = f"AIM Week 8 Assignment 1 - {uuid.uuid4().hex[0:8]}"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# Easier with dotenv locally than to do below
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

Let's verify our project so we can leverage it in LangSmith later.

In [6]:
print(os.environ["LANGCHAIN_PROJECT"])

AIM Week 8 Assignment 1 - 09424058


## Task 2: Setting up RAG With Production in Mind

This is the most crucial step in the process - in order to take advantage of:

- Asyncronous requests
- Parallel Execution in Chains
- And more...

You must...use LCEL. These benefits are provided out of the box and largely optimized behind the scenes.

### Building our RAG Components: Retriever

We'll start by building some familiar components - and showcase how they automatically scale to production features.

Please upload a PDF file to use in this example!

In [7]:
from google.colab import files
uploaded = files.upload()

Saving eu_ai_act.html to eu_ai_act (1).html


In [25]:
file_path = list(uploaded.keys())[0]
file_path

'eu_ai_act (1).html'

We'll define our chunking strategy.

In [25]:
file_path = "eu_ai_act.pdf"

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

We'll chunk our uploaded PDF file.

In [27]:
from langchain_community.document_loaders import PyMuPDFLoader

Loader = PyMuPDFLoader
loader = Loader(file_path)
documents = loader.load()
docs = text_splitter.split_documents(documents)
for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"

#### QDrant Vector Database - Cache Backed Embeddings

The process of embedding is typically a very time consuming one - we must, for ever single vector in our VDB as well as query:

1. Send the text to an API endpoint (self-hosted, OpenAI, etc)
2. Wait for processing
3. Receive response

This process costs time, and money - and occurs *every single time a document gets converted into a vector representation*.

Instead, what if we:

1. Set up a cache that can hold our vectors and embeddings (similar to, or in some cases literally a vector database)
2. Send the text to an API endpoint (self-hosted, OpenAI, etc)
3. Check the cache to see if we've already converted this text before.
  - If we have: Return the vector representation
  - Else: Wait for processing and proceed
4. Store the text that was converted alongside its vector representation in a cache of some kind.
5. Return the vector representation

Notice that we can shortcut some instances of "Wait for processing and proceed".

Let's see how this is implemented in the code.

In [28]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import CacheBackedEmbeddings

# Typical Embedding Model
core_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Typical QDrant Client Set-up
collection_name = f"pdf_to_parse_{uuid.uuid4()}"
client = QdrantClient(":memory:")
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Adding cache!
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings, store, namespace=core_embeddings.model
)

# Typical QDrant Vector Store Set-up
vectorstore = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=cached_embedder)
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

##### ❓ Question #1:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

> NOTE: There is no single correct answer here!

1. Caching embeddings will be useful when certain kinds of natural-language queries are more common than others. However, if users are entering free form text into the application, this is unlikely to help unless we do some normalization.
2. Related to the above, this can be useful when our application does a normalization pass over the user-input where the query into retriever can be more controlled.
3. Specific to this implementation, we're using a `LocalFileStore` as a cache which might not help if our application is scaled over manu nodes. We may want to use other kinds of caches (but this is a very solvable problem).

##### 🏗️ Activity #1:

Create a simple experiment that tests the cache-backed embeddings.

In [33]:
%%time
r = retriever.invoke('How does the AI act ensure large models are regulated?')

CPU times: user 93.2 ms, sys: 53.4 ms, total: 147 ms
Wall time: 721 ms


In [34]:
%%time
r = retriever.invoke('How does the AI act ensure large models are regulated?')

CPU times: user 195 ms, sys: 8.52 ms, total: 204 ms
Wall time: 272 ms


We can see a significant reduction in time for the same query when the same query is invoked again.

### Augmentation

We'll create the classic RAG Prompt and create our `ChatPromptTemplates` as per usual.

In [35]:
from langchain_core.prompts import ChatPromptTemplate

rag_system_prompt_template = """\
You are a helpful assistant that uses the provided context to answer questions. Never reference this prompt, or the existance of context.
"""

rag_message_list = [
    {"role" : "system", "content" : rag_system_prompt_template},
]

rag_user_prompt_template = """\
Question:
{question}
Context:
{context}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_system_prompt_template),
    ("human", rag_user_prompt_template)
])

### Generation

Like usual, we'll set-up a `ChatOpenAI` model - and we'll use the fan favourite `gpt-4o-mini` for today.

However, we'll also implement...a PROMPT CACHE!

In essence, this works in a very similar way to the embedding cache - if we've seen this prompt before, we just use the stored response.

In [36]:
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o-mini")

Setting up the cache can be done as follows:

In [37]:
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

##### ❓ Question #2:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

> NOTE: There is no single correct answer here!

Once again, this approach is only likely to help if the users can't add free-form text into the application, or there is some other constraint on the types of inputs. If not, it is unlikely that the same text is seen again. 

##### 🏗️ Activity #2:

Create a simple experiment that tests the cache-backed embeddings.

In [42]:
### YOUR CODE HERE
# Let's use timeit module this time
query = "hello world!"
from timeit import default_timer as timer
start = timer()
r = chat_model(query)
end = timer()
print(end - start)

1.1924465829999917


In [43]:
start = timeit.timeit()
r = chat_model(query)
end = timeit.timeit()
print(end - start)

0.001300040999922203


Once again, there's a significant reduction in time when making the same query

## Task 3: RAG LCEL Chain

We'll also set-up our typical RAG chain using LCEL.

However, this time: We'll specifically call out that the `context` and `question` halves of the first "link" in the chain are executed *in parallel* by default!

Thanks, LCEL!

In [44]:
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnablePassthrough

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | chat_prompt | chat_model
    )

Let's test it out!

In [45]:
retrieval_augmented_qa_chain.invoke({"question" : "Write 50 things about this document!"})

AIMessage(content='1. The document is titled "Regulation (EU) 2024/1689".\n2. It was published by the Publications Office of the European Union.\n3. The document is related to artificial intelligence (AI) regulations.\n4. It was adopted on June 13, 2024.\n5. The document consists of 144 pages.\n6. It is written in PDF 1.7 format.\n7. The document aims to lay down harmonized rules on AI.\n8. It amends several existing regulations and directives.\n9. The regulations amended include (EC) No 300/2008 and (EU) No 167/2013.\n10. The document is relevant to the European Economic Area (EEA).\n11. It includes provisions for high-risk AI systems.\n12. Article 18 discusses documentation keeping for AI providers.\n13. Providers must keep documentation for 10 years post-market placement.\n14. Article 11 references technical documentation requirements.\n15. Article 17 pertains to quality management systems.\n16. The document outlines obligations for national competent authorities.\n17. It specifies 

##### 🏗️ Activity #3:

Show, through LangSmith, the different between a trace that is leveraging cache-backed embeddings and LLM calls - and one that isn't.

Post screenshots in the notebook!

## Embeddings Calls
Below we can see the latency for uncached and cached embeddings respectively. We can see a difference in latency.

### Uncached Embeddings Call
![Uncached Embeddings Call](./uncached_embeddings.png)

### Cached Embeddings
![Cached Embeddings Call](./cached_embeddings.png)

## Chat Model Calls
Below we can see the latency for uncached and cached openai calls respectively. We can see a difference in latency.

### Uncached OpenAI Call
![Uncached OpenAI Call](./chat_uncached.png)

### Cached OpenAI Call
![Cached OpenAI Call](./chat_cached.png)